In [30]:
from pathlib import Path
import matplotlib.pyplot as plt

from docling.document_converter import DocumentConverter
from docling.document_converter import PdfFormatOption
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions

from docling_core.types.doc.base import ImageRefMode
from docling_core.types.doc.document import PictureItem
from docling_core.types.doc.document import TableItem
from docling_core.types.doc.document import TextItem
from docling_core.types.doc.document import DoclingDocument

output_dir = Path('Output')
output_dir.mkdir(exist_ok = True)

In [13]:
converter = DocumentConverter()
source = 'https://arxiv.org/pdf/2410.23335.pdf'
result = converter.convert(source)

2025-11-05 10:23:22,803 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-11-05 10:23:22,805 - INFO - Going to convert document batch...
2025-11-05 10:23:22,805 - INFO - Initializing pipeline for StandardPdfPipeline with options hash f9730ffaa6e7f8d4fb0c98c8df3f18cb
2025-11-05 10:23:22,806 - INFO - rapidocr cannot be used because onnxruntime is not installed.
2025-11-05 10:23:22,806 - INFO - easyocr cannot be used because it is not installed.
2025-11-05 10:23:22,806 - INFO - Accelerator device: 'cuda:0'
[INFO] 2025-11-05 10:23:22,816 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-11-05 10:23:22,824 [RapidOCR] download_file.py:60: File exists and is valid: /home/torben/Huginn/.venv/lib/python3.12/site-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-11-05 10:23:22,825 [RapidOCR] torch.py:54: Using /home/torben/Huginn/.venv/lib/python3.12/site-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-11-05 10:23:22,912 [RapidOCR] base.py:22: Using

In [ ]:
print(result.document.export_to_doctags())